In [29]:
from Scripts.import_data import *
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error

In [30]:
balance_data = import_csv_dict_of_dataframes('./Data/companies_balance_sheet.csv')

In [31]:
concated_balance_data = pd.concat(balance_data.values())

Balance_data to słownik, którego klucze to nazwy spółek natomiast wartości do dataframe'y z danymi od 2017 roku do 2021 roku. 

In [41]:
poi = ['Revenues from sales', 'Operating profit/loss', 'Net profit/loss attributable to equity holders of the parent', 
       'Total comprehensive income attributable to equity holders', 'Depreciation', 'Cash flow from operating activities', 
       'Cash flow from investing activities', 'Cash flow from financing activities', 'Aktywa', 'Non-current assets',
       'Current assets', 'Equity shareholders of the parent', 'Non-current liabilities', 'Current liabilities', 
       "Number of shares ('000)", 'Earnings per share', 'Total comprehensive income per share', 'Book value per share']

X = concated_balance_data[concated_balance_data.index.get_level_values(1).isin(poi)]


# tutaj daj coś do y


In [33]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

models = {
    'Linear Regression': LinearRegression(),
    'Random Forest': RandomForestRegressor(),
    'Gradient Boosting': GradientBoostingRegressor(),
    'Support Vector Machine': SVR(kernel='linear'),
    'Neural Network': MLPRegressor(hidden_layer_sizes=(100,), activation='relu', solver='adam', max_iter=1000)
}

# Train and evaluate each model
for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    print(f'{name} MSE: {mse}')

# Pick the best performing model for deployment


NameError: name 'y' is not defined